# Model architecture

The model is built in the function `NN` below. The model can be customized with various arguments, but the general structure is as follows:

* **Text Vectorizer**: this encodes the input string to a sequence of integers. The layer will produce sequences of length up to 100, and each *word* is encoded as an integer. Hence input strings should ideally have length at most 100 to avoid lost information. The size of the vocabulary can be customized.
* **Embedding layer**: the input sequences are embedded into $n$-dimensional space. You may choose $n$, and you may also try using embeddings pretrained using GloVe (for `embed_layer` enter either "glove 50" or "glove 50 trainable" for the 50 dimensional embeddings, making this layer either trainable or not. GloVe embeddings may be 50, 100, 200 or 300 dimensional.)
* **Dropout layer**: the dropout rate can be set with by `Dropout_rate`. The same rate is used for all dropout layers.
* **Convolutional layers**: by default none are used. The number of filters and filter size can be customized. If used, the `Conv1D` layer is followed by a **max pooling** layer (the pooling size can be customized) and a **dropout layer**.
* **LSTM layers**: you may choose how many layers, how many unites and whether to use kernel regularization. You may also choose whether you want the layers to be bidirectional or not (with the `BRNN` argument).
* **Dropout layer**
* **Dense layers**: you may choose how many layers to use, and how many neurons in each layer. You may also set the regularization and whether to use batch normalization.
* **Dropout layer**
* **Output layer**: one neuron with sigmoid activation to output classification probabilities.

The `TextVectorizer` layer needs to be initialized first, and is done so with the `vect_layer_maker` function. It is here where you can choose the vocabulary size.

If using the GloVe embeddings, then these need to be initialized too. Use the `get_glove_embedding_layer` to do this, which returns a pretrained embedding layer.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Activation, BatchNormalization, TextVectorization, Dropout, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.regularizers import l2
from time import time
from functools import partial
from zipfile import ZipFile
import re
from helper_functions import * # functions defined in helper_functions.py

In [ ]:
def vect_layer_maker(X_train, vocab_size=10000, sequence_length=100):
    vectorize_layer = TextVectorization(max_tokens = vocab_size,
                                        output_mode='int',
                                        output_sequence_length=100,
                                        name='vectorizer'
                                       )
    vectorize_layer.adapt(X_train)
    return vectorize_layer

In [ ]:
def get_glove_embedding_layer(vectorizer_layer, embed_dim=300, trainable=False):
    # First get the vectors            
    glove_dict = {}
    if embed_dim not in [50,100,200,300]:
        raise ValueError('Glove embedding dimensions are 50, 100, 200 or 300 only.')
    filename = f'../input/glove6b/glove.6B.{embed_dim}d.txt'
    with open(filename) as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = values[1:]
            glove_dict[word] = np.asarray(vector, dtype='float32')
    # Create vocab dictionary from the vectorizer layer
    vocab_list = vectorizer_layer.get_vocabulary()
    vocab_dict = {word : index for index, word in enumerate(vocab_list)}
    # Next create the matrix
    embedding_matrix = np.zeros((len(vocab_dict)+1, embed_dim))
    for word, index in vocab_dict.items():
        embedding_vector = glove_dict.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    # Now create and return the layer
    return Embedding(input_dim = len(vocab_dict)+1,
                     output_dim = embed_dim,
                     trainable = trainable,
                     weights = [embedding_matrix]
                    )

In [ ]:
def NN(embed_layer = 32,
       Conv_filters=None,
       Conv_filter_size=3,
       Conv_pool_size=2,
       LSTM_layers=2, 
       LSTM_units=64, 
       LSTM_kernel_reg=None,
       BRNN=False,
       Dense_layers=1, 
       Dense_units=128, 
       Dense_reg=None,
       Batch_norm=False,
       Dropout_rate=0,
       learning_rate=0.001
      ):
    '''
    Builds and compiles RNN.
    
    Args:
        embed_layer: int (dimension of embedding) or pre-built keras layer
        LSTM_layers: int  -- number of LSTM layers, come after the embedding layer
        LSTM_units: int
        LSTM_kernel_reg:  tf.keras.regularizers object for regularization of input weights
        Dense_layers: int  -- number of Dense layers afer LSTM
        Dense_units: int
        Dense_reg: tf.keras.regularizers object for regularization of weights
        Dropout_rate: float (default 0 = no dropout)  -- dropout rate for dropout layers
        
    Returns:
        compiled model
    '''
    model = Sequential()
    # vectorization layer
    model.add(vectorize_layer)
    # Embedding layer
    if isinstance(embed_layer, int):
        embed_layer = Embedding(vocab_size+1, embed_layer)
    else:
        model.add(tf.keras.layers.Input(shape=[100], dtype=tf.string))
        if embed_layer in [f'glove {x}' for x in [50,100,200,300]]:
            embed_dim = int(re.findall(r'\d+', embed_layer)[0])
            embed_layer = get_glove_embedding_layer(vectorize_layer, embed_dim=embed_dim, trainable=False)
        elif embed_layer in [f'glove {x} trainable' for x in [50,100,200,300]]:
            embed_dim = int(re.findall(r'\d+', embed_layer)[0])
            embed_layer = get_glove_embedding_layer(vectorize_layer, embed_dim=embed_dim, trainable=True)
        else:
            raise ValueError('Embedding layer format not recognized. Should be either integer for trainable embedding of that dimension, or either "glove x" or "glove x trainable", for x = 50, 100, 200, 300')
    model.add(embed_layer)
    # Dropout layer
    model.add(Dropout(Dropout_rate))
    # Add Conv layer (optional)
    if Conv_filters != None:
        model.add(Conv1D(filters=Conv_filters, kernel_size=Conv_filter_size, padding = 'same', activation='relu', name='Conv1D'))
        model.add(MaxPooling1D(pool_size=Conv_pool_size, name='MaxPool'))
        model.add(Dropout(Dropout_rate))
    # add LSTM layers
    if BRNN:
        if LSTM_layers>1:
            for i in range(LSTM_layers-1):
                model.add(Bidirectional(LSTM(LSTM_units, return_sequences = True, kernel_regularizer = LSTM_kernel_reg, name = f'LSTM_{i}')))
        else:
            i=0
        model.add(Bidirectional(LSTM(LSTM_units, name = f'LSTM_{i+1}')))
        
    else:
        if LSTM_layers>1:
            for i in range(LSTM_layers-1):
                model.add(LSTM(LSTM_units, return_sequences = True, kernel_regularizer = LSTM_kernel_reg, name = f'LSTM_{i}'))
        else:
            i=0
        model.add(LSTM(LSTM_units, name = f'LSTM_{i+1}'))
    # Another dropout layer
    model.add(Dropout(Dropout_rate))
    # add fully connected layers
    for i in range(Dense_layers):
        model.add(Dense(Dense_units, kernel_initializer = 'he_uniform', activation=None, kernel_regularizer=Dense_reg, name = f'Dense_{i}'))
        if Batch_norm:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
    # Another dropout layer
    model.add(Dropout(Dropout_rate))
    # add final layer, outputting proability speech is Tory
    model.add(Dense(1, activation='sigmoid', name = 'output_layer'))
    # compile
    Adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])
    
    return model

# The grid search begins...

In [ ]:
train_ds, val_ds, test_ds = load_data(batch_size=512) # load the data

vocab_size=30000
vectorize_layer = vect_layer_maker(train_ds.map(lambda x,y:x), vocab_size=vocab_size, sequence_length=100)

In [ ]:
reg = l2(0.01)
reg2 = l2(0.1)
n_repeats = 1
history_log = grid_search(NN, train_ds, val_ds, n_repeats = n_repeats, n_epochs=10, early_stopping_patience=6,verbose=1,
                                   embed_layer = [16],
                                   Conv_filters=[16],
                                   Conv_filter_size=[5],
                                   Conv_pool_size=[2],
                                   LSTM_layers=[2], 
                                   LSTM_units=[32], 
                                   LSTM_kernel_reg=[reg2],
                                   BRNN=[True],
                                   Dense_layers=[2], 
                                   Dense_units=[64], 
                                   Dense_reg=[reg2],
                                   Batch_norm=[True],
                                   Dropout_rate=[0.3],
                                   learning_rate=[0.0001]
                                  )

In [ ]:
visualize_log(history_log)